This is a notebook for the capstone project

In [2]:
import pandas as pd
import numpy as np

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Final project starts here

## Neighbourhood data collection

### Installing and importing relevant packages 

In [4]:
# !conda install -c anaconda beautifulsoup4 

In [5]:
import requests
import lxml
from bs4 import BeautifulSoup as bs
import pandas as pd

### Scrape the Toronto wiki page and create dataframe

In [6]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(wiki_url)

In [7]:
html = page.text
soup = bs(html,'lxml')
table = soup.find('table')
table
data=[]
for th in table.find_all('tr'):
    data.append(th.text.strip('\n').split('\n'))

In [8]:
df = pd.DataFrame.from_records(data,columns = data[0])
df.drop(0,axis=0,inplace=True)

### Drop "not assigned" boroughs

In [9]:
df.drop(df[df.Borough=='Not assigned'].index,axis=0,inplace=True)
# df
df.Neighbourhood[df.Neighbourhood=='Not assigned']=df.Borough[df.Neighbourhood=='Not assigned']
df.reset_index(drop=True,inplace=True)
df.shape

(212, 3)

### Merge neighbourhoods within the same postcode

In [10]:
df1 = df.groupby('Postcode')['Neighbourhood'].apply(lambda x:'%s'%', '.join(x))
df = df.merge(df1.to_frame().reset_index(),on='Postcode',how='left',suffixes=('_left', '_right'))
df.rename(columns={'Neighbourhood_right': 'Neighbourhood'},inplace=True)
df.drop('Neighbourhood_left',axis=1,inplace=True)
df.drop_duplicates(subset='Postcode',inplace=True)
df.shape

(103, 3)

In [11]:
# !pip install geocoder

### The geocoder does not work properly. Using the given .csv file instead

import geocoder

def get_lat_lon(row):
    import geocoder # import geocoder

    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(row['Postcode']))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude, longitude

### Download the csv file and merge with neighbourhood data

In [12]:
!wget -q -O 'toronto_postcodes.csv' https://cocl.us/Geospatial_data

In [13]:
df_postcodes = pd.read_csv('toronto_postcodes.csv')
df_postcodes.shape

(103, 3)

In [14]:
df = df.merge(df_postcodes,left_on='Postcode',right_on='Postal Code',how='left')
df.drop('Postal Code', axis=1, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


## Venue data collection

In [ ]:
# The code was removed by Watson Studio for sharing.

## Data exploration

In [17]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  55.52 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  34.88 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  38.85 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  48.30 MB/s
Libraries imported.


### Map of neighbourhoods in Toronto

In [18]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [37]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Just downtown Toronto

In [33]:
# df_toronto = df[df['Borough']]
df_toronto = df[df['Borough']=='Downtown Toronto']
df_toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
36,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
42,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
48,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817


In [36]:
# create map of Toronto using latitude and longitude values
map_dt_toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dt_toronto)  
    
map_dt_toronto